In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling Main.sc


path: String = "/home/kunmok/Git/chisel-bootcamp/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

In [110]:
class MyOperators extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(2.W))
    val in2     = Input(UInt(3.W))
    val in3     = Input(SInt(1.W))
    val out_tmp1= Output(UInt(2.W))
    val out_tmp2= Output(UInt(log2Ceil(5).W))
    val out_tmp3= Output(SInt(1.W))
  })

  io.out_tmp1 := io.in + 2.U 
  io.out_tmp2 := io.in2 + 2.U
  io.out_tmp3 := io.in3 
  // (3+1 -> 4 -> 0, 3+2 -> 5 -> 1)
}

class MyOperatorsTester(c: MyOperators) extends PeekPokeTester(c) {
  poke(c.io.in, 3)
  expect(c.io.out_tmp1, 1)
  poke(c.io.in2, 3)
  expect(c.io.out_tmp2, 5)
  poke(c.io.in3, -1)
  expect(c.io.out_tmp3, -1)

}
assert(Driver(() => new MyOperators) {c => new MyOperatorsTester(c)})
println("SUCCESS!!")

[info] [0.000] Elaborating design...
[info] [0.005] Done elaborating.
Total FIRRTL Compile Time: 5.3 ms
Total FIRRTL Compile Time: 4.9 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1540720459239
test cmd109WrapperHelperMyOperators Success: 3 tests passed in 5 cycles taking 0.003010 seconds
[info] [0.001] RAN 0 CYCLES PASSED
SUCCESS!!


defined class MyOperators
defined class MyOperatorsTester

In [51]:
  def dec2bitarray(genPolynoimial: List[Int], bit_width: Int) = {
    val numRow = genPolynoimial.length
    val bitarray = Array.ofDim[Int](numRow, bit_width)
//     val binary_string = (i: Int) => {genPolynoimial(i).toBinaryString}
    for (i <- 0 until numRow){
      val binary_string = genPolynoimial(i).toBinaryString
      val len = binary_string.length
      for (j <- 0 until bit_width){
        bitarray(i)(bit_width-j-1) = binary_string(len-j-1).asDigit
      }
    }
    bitarray
  }

  def findMinBitWidth(arg: List[Int]): Int = {
    val result: Int = math.ceil(math.log10(arg.max)/math.log10(2)).asInstanceOf[Int]
    result
  }

  def onesInPuncMat(arg: Array[Array[Int]]): Unit ={
    arg.map(breeze.linalg.Vector(_)).reduce(_ + _)
  }

val binary_string = (x: Int, arg: List[Int]) => { arg(x).toBinaryString }

println(binary_string(0, List(1,1,0)))

1


defined function dec2bitarray
defined function findMinBitWidth
defined function onesInPuncMat
binary_string: (Int, List[Int]) => String = <function2>

In [40]:
class MyOperators2 extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(2.W))
    val out_tmp1= Output(UInt(2.W))
    val out_arr = Output(Vec(2, Vec(3, UInt(1.W))))
  })

    io.out_tmp1 := io.in + 2.U 
    val genPolynomial = List(7, 5)
    val K = 3
    val n = 2 
    val genPolyList       = dec2bitarray(genPolynomial, K)
    val genPolyVec        = Wire(Vec(n, Vec(K, UInt(1.W))))

    for (x <- 0 until n){ 
      for (y <- 0 until K){
        genPolyVec(x)(y) := (genPolyList(x)(y)).U 
      }
    }
    
//     (0 until n).map(x => {
//     (0 until K).map(y => {
//       genPolyVec(x)(y)  := (genPolyList(x)(y)).U
//       })
//     })
    for (x <- 0 until n){
        for (y <- 0 until K){
            io.out_arr(x)(y) := genPolyVec(x)(y)
        }
    }
}

class MyOperatorsTester2(c: MyOperators2) extends PeekPokeTester(c) {
  poke(c.io.in, 3)
  expect(c.io.out_arr(0)(0), 1)
    expect(c.io.out_arr(0)(1), 1)
    expect(c.io.out_arr(0)(2), 1)
    expect(c.io.out_arr(1)(0), 1)
    expect(c.io.out_arr(1)(1), 0)
    expect(c.io.out_arr(1)(2), 1)
}
assert(Driver(() => new MyOperators2) {c => new MyOperatorsTester2(c)})
println("SUCCESS!!")

[info] [0.000] Elaborating design...
[info] [0.085] Done elaborating.
Total FIRRTL Compile Time: 19.6 ms
Total FIRRTL Compile Time: 17.0 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1540710168070
test cmd39WrapperHelperMyOperators2 Success: 6 tests passed in 5 cycles taking 0.006581 seconds
[info] [0.005] RAN 0 CYCLES PASSED
SUCCESS!!


defined class MyOperators2
defined class MyOperatorsTester2

In [89]:
class MyOperators3 extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(1.W))
    val out     = Output(Vec(3, UInt(1.W)))
    val erase   = Input(UInt(1.W))
  })

    val K = 3
    val n = 2 
    val m = 2 
    
    val shiftReg          = RegInit(VecInit(Seq.fill(K)(0.U(1.W)))) // initialze memory with all zeros
    val wireGND = Wire(Vec(K, UInt(1.W)))
    wireGND := VecInit(Seq.fill(K)(0.U(1.W)))
//     (0 to m).map(i => {wireGND(i) := 0.U})    // this also works 
    shiftReg(0) := io.in                         // receive input from FIFO. I probably need to use MUX
    (1 to m).reverse.map(i => {shiftReg(i) := shiftReg(i-1)})
    when(io.erase === 1.U){
//         shiftReg := VecInit(Seq.fill(K)(0.U(1.W)))
//         (0 to m).map(i => {shiftReg(i) := 0.U})
        shiftReg := wireGND
    }
    io.out := shiftReg 
    
}

class MyOperatorsTester3(c: MyOperators3) extends PeekPokeTester(c) {
    poke(c.io.in, 1)
    poke(c.io.erase, 0)
    step(1) 
    expect(c.io.out(0), 1)
    expect(c.io.out(1), 0)
    expect(c.io.out(2), 0)
    step(1)
    expect(c.io.out(0), 1)
    expect(c.io.out(1), 1)
    expect(c.io.out(2), 0)
    step(1)
    expect(c.io.out(0), 1)
    expect(c.io.out(1), 1)
    expect(c.io.out(2), 1)
    step(1)
    poke(c.io.erase, 1)
    step(1)
    step(1)
    expect(c.io.out(0), 0)
    expect(c.io.out(1), 0)
    expect(c.io.out(2), 0)
}
assert(Driver(() => new MyOperators3) {c => new MyOperatorsTester3(c)})
println("SUCCESS!!")

[info] [0.000] Elaborating design...
[info] [0.005] Done elaborating.
Total FIRRTL Compile Time: 5.4 ms
Total FIRRTL Compile Time: 5.5 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1540717781515
test cmd88WrapperHelperMyOperators3 Success: 12 tests passed in 11 cycles taking 0.003361 seconds
[info] [0.003] RAN 6 CYCLES PASSED
SUCCESS!!


defined class MyOperators3
defined class MyOperatorsTester3